# Notebook to select host galaxies and submit candidates to TNS

In [1]:
import sys, os
import pandas as pd
import numpy as np
import ipyaladin as ipyal
from ipywidgets import Layout, Box, widgets
import json
import time
import pickle

In [2]:
from astroquery.ned import Ned
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.table import Table, Column

In [3]:
sys.path.append("./lib")
from alerce_tns import *

In [4]:
from delight.delight import Delight

In [5]:
# continuously load modules (for easier debugging)
%load_ext autoreload
%autoreload 2

# Start API

In [6]:
alerce = alerce_tns()

# Load recent candidates

**WARNING: CHECK THE DATE**

In [7]:
refstring = "20220716"
candidates_dir = "candidates"
candidates = open("%s/%s.csv" % (candidates_dir, refstring), encoding='utf-8-sig').read().splitlines()
if "" in candidates:
    candidates.remove("") 
candidates = np.sort(np.unique(np.array([cand.strip() for cand in candidates])))
print(candidates, len(candidates))

['ZTF22aapuazu' 'ZTF22aarycqo' 'ZTF22aasvuof' 'ZTF22aasxgjp'
 'ZTF22aasxnjm' 'ZTF22aatgiuw' 'ZTF22aatgjnu' 'ZTF22aatkeyf'
 'ZTF22aatlqxr' 'ZTF22aatnyqk' 'ZTF22aatrbvj' 'ZTF22aatrnbl'
 'ZTF22aatrpse' 'ZTF22aatrujl' 'ZTF22aatrzyq' 'ZTF22aatsbjo'
 'ZTF22aatsdyx' 'ZTF22aatsfly' 'ZTF22aattadv' 'ZTF22aattzza'
 'ZTF22aatunnt' 'ZTF22aatuxpc' 'ZTF22aatvbyz' 'ZTF22aatvkqs'] 24


In [8]:
stats = alerce.query_objects(oid=candidates, format='pandas', page_size=len(candidates))
delight_datadir = "delight_data"
dclient = Delight(delight_datadir, candidates, stats.meanra.to_numpy(), stats.meandec.to_numpy())
dclient.download()
dclient.get_pix_coords()
dclient.compute_multiresolution(nlevels=5, domask=False, doobject=True, doplot=False)
model_dir = "/home/fforster/Dropbox/Work/DELIGHT/data/model"
model_name = "DELIGHT_v1.h5"
dclient.load_model(model_dir, model_name)
print("Preprocessing")
dclient.preprocess()
print("Predicting")
dclient.predict()

Loading WCS information
Preprocessing
Predicting


In [9]:
#for oid in candidates:
#    dclient.plot_host(oid)

# Select host galaxies using aladin widget

## Do automatic selection

In [10]:
bestguess = dclient.df[["ra_delight", "dec_delight"]].rename({"ra_delight": "ra", "dec_delight": "dec"}, axis=1)
alerce.bestguess = bestguess

In [11]:
alerce.select_hosts(candidates, refstring, ned=True, simbad=True, catsHTM=False, vizier=False)

Loading and skipping already saved hosts...
hosts/20220716_hosts.csv
Cannot load galaxy information, creating new information.


Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

/home/fforster/anaconda3/lib/python3.8/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '1:05:42.656 -0:53:20.6534': No astronomical object found :
  warnings.warn("Warning: The script line number %i raised "


,host_name,host_ra,host_dec,host_offset,host_offset_bestguess,bestguess_offset,host_source,host_redshift_spec,host_redshift,host_redshift_error,host_redshift_type
oid,,,,,,,,,,,
ZTF22aapuazu,SDSS J010542.66-005320.8,16.42774,-0.889133,0.159994,0.225378,0.132016,SDSSDR16,False,0.143959,0.028953,photoz


23 candidates remaining


...

/home/fforster/anaconda3/lib/python3.8/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '21:46:41.5121 +84:00:29.8978': No astronomical object found :
  warnings.warn("Warning: The script line number %i raised "


,host_name,host_ra,host_dec,host_offset,host_offset_bestguess,bestguess_offset,host_source,host_redshift_spec,host_redshift,host_redshift_error,host_redshift_type
oid,,,,,,,,,,,
ZTF22aatvkqs,WISEA J214641.85+840030.0,326.67634,84.00794,2.209391,1.825568,1.580997,NED,False,NaN,NaN,NaN


0 candidates remaining


Summary of host galaxies:
Saving hosts to hosts/20220716_hosts.csv


,host_name,host_ra,host_dec,host_offset,host_offset_bestguess,bestguess_offset,host_source,host_redshift_spec,host_redshift,host_redshift_error,host_redshift_type
oid,,,,,,,,,,,
ZTF22aatvkqs,WISEA J214641.85+840030.0,326.67634,84.00794,2.209391,1.825568,1.580997,NED,False,NULL,NULL,NULL
ZTF22aatvbyz,SDSS J020004.38+084456.3,30.018236,8.748997,0.633893,0.298643,0.932202,SDSSDR16,False,0.126866,0.034842,photoz
ZTF22aatuxpc,SDSS J233255.62+270921.3,353.231733,27.155923,2.776546,0.702978,3.119576,SDSSDR16,False,0.072873,0.029296,photoz
ZTF22aatunnt,SDSS J214319.13+053451.0,325.829698,5.580847,1.443362,0.501268,1.353908,SDSSDR16,False,0.083412,0.032715,photoz
ZTF22aattzza,WISEA J185220.10+435453.4,283.08375,43.91481,1.436774,0.560704,1.179427,NED,False,0.092962,NULL,PHOT
ZTF22aattadv,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
ZTF22aatsfly,SDSS J174111.16+461446.6,265.296508,46.24628,2.213072,0.456695,1.760662,SDSSDR16,False,0.128017,0.01283,photoz
ZTF22aatsdyx,SDSS J172055.14+294426.4,260.229768,29.740685,3.177645,0.492974,2.895159,SDSSDR16,False,0.149418,0.024392,photoz
ZTF22aatsbjo,SDSS J164929.40+224916.7,252.372507,22.821329,1.39677,0.312795,1.17464,SDSSDR16,False,0.138089,0.038266,photoz


Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv
Saving hosts to hosts/20220716_hosts.csv


## Do manual selections for errors

Errors could happen because of an error in DELIGHT or because the association favoured a more distant host with redshift.

In [ ]:
errors = []#"ZTF22aasxnjm", "ZTF22aatrnbl", "ZTF22aattadv"] #"ZTF22aaujkdb", "ZTF22aaveghe"]
# confirm if this was a problem with DELIGHT
#for oid in errors:
#    dclient.plot_host(oid)
# do manual selection
if errors != []:
    alerce.select_hosts(errors, refstring, ned=True, simbad=True, catsHTM=False, vizier=False, dobestguess=False)

## Manual modifications

In [ ]:
#oid = "ZTF21acbcfmh" #ZTF21abidhcz"
##oid = 'ZTF20aclzygy'#ZTF20acgigfo'
##oid = "ZTF20aaqrmxg" #"ZTF20aakdppm" #ZTF19acxrihd"
#oid = 'ZTF22aaajgss' #ZTF21aaryakx' #ZTF20acvbneg'
#alerce.candidate_hosts.host_name[oid] = "M31"
#alerce.candidate_hosts.host_ra[oid] = ""
#alerce.candidate_hosts.host_dec[oid] = ""
#alerce.candidate_hosts.host_offset[oid] = ""
#alerce.candidate_hosts.host_source[oid] = "CDS"
#alerce.candidate_hosts.host_redshift_spec[oid] = True
#alerce.candidate_hosts.host_redshift[oid] = = "0.001341"
#alerce.candidate_hosts.host_redshift_error[oid] = "0.000040"
#alerce.candidate_hosts.host_redshift_type[oid] = "specz"
# #"nan" #"0.041" #nan"
##for i in list(alerce.candidate_hosts):
##    alerce.candidate_hosts.loc['ZTF21aaryakx'][i] = "NULL"

## Save hosts

In [ ]:
alerce.candidate_hosts.to_csv("hosts/%s_hosts.csv" % refstring)
alerce.candidate_hosts.head(100)

## Show nearest sources (z < 0.01)

In [ ]:
zth = 0.1
nearest = pd.to_numeric(alerce.candidate_hosts.loc[alerce.candidate_hosts.host_redshift != "NULL"].host_redshift).sort_values()
nearest = nearest.loc[nearest < zth]
for i in ["https://alerce.online/object/%s %s" % (i, alerce.candidate_hosts.host_redshift.loc[i]) for i in nearest.index]:
    print(i)

# Prepare TNS report

## Sandbox or real

In [ ]:
sandbox = False #False # True: test in sandbox, False: send to real website

## Get API key and url

In [ ]:
if sandbox:
    print("Getting API sandbox key...")
    api_key = open("API_sandbox.key", "r").read()
    api_key = open("API.key", "r").read() # sandbox key appears not to be working
else:
    print("Getting API key...")
    api_key = open("API.key", "r").read()
    
if sandbox:
    url_tns_api = "https://sandbox.wis-tns.org/api" #"https://www.wis-tns.org/" #"https://sandbox-tns.weizmann.ac.il/api"
else:
    url_tns_api = "https://www.wis-tns.org/api" #"https://wis-tns.weizmann.ac.il/api" #"https://www.wis-tns.org/" #"https://wis-tns.weizmann.ac.il/api"
    
if not sandbox:
    print("WARNING: submitting to real (not sandbox) TNS website")

## Confirm reporter names

This is also used for Skyportal

In [ ]:
reporter = "F. Forster, F.E. Bauer, G. Pignata, A. Munoz-Arancibia, A. Mourao, R. Dastidar, L. Hernandez-Garcia, L. Galbany, E. Camacho, J. Silva-Farfan, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker"
#F. Forster, F.E. Bauer, A. Munoz-Arancibia, G. Pignata, L. Hernandez-Garcia, L. Galbany, E. Camacho, J. Silva-Farfan, A. Mourao, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker"

## Build the report

In [ ]:
# first author: the person submitting, 2nd author: those that vetoed, then everyone else
tns_report = {"at_report": {}}
counter = 0
for oid in candidates[::-1]: # invert to report old candidates first
    #if oid in ["ZTF20aauqmwf"]:#["ZTF19acgjpfd", "ZTF18acszatf", "ZTF18acrkmnm"]:
    #    continue
    #if oid in ["ZTF20acqnfkh ", "ZTF20acneabh"]:#ZTF20acplctd":#aamtgd": #ZTF19acymtgr": #ZTF19acnqqhl":#in ["ZTF19acmdwjy", "ZTF19acmdwjz"]:
    #    continue
    #if oid == "ZTF20ablnjwh": #ZTF20abkmxrm":#ZTF20abgfgne":
    #    continue
    if oid in ["ZTF22aabrskg"]: #ZTF21abgvidd"]: #ZTF21aawgsfp"]:#ZTF21aalcrjh"]: #ZTF21aaanwxo"]:#, ZTF20adadixi"]:# ZTF20acsggey"]:#, "ZTF20acmevcc", "ZTF20acmctng", "ZTF20acmdvzk"]: #== "ZTF20acgknep":#"ZTF20acdcdqq": #ZTF20acaehey": #ZTF20abyjoxb":#"ZTF20abqjddh":#ZTF20abkxqzv":#ZTF20abgbfyk":
        continue

    print("\n", counter, oid)
    sn_report = alerce.do_TNS_report(api_key, oid, reporter, verbose=False, test=False)
    if sn_report:# or oid == "ZTF20abyptpc":
        tns_report["at_report"]["%i" % counter] = sn_report
        counter += 1


## Manual modifications

In [ ]:
#report['at_report']['0']['remarks'] = "Fast rising transient (g-rise > 1.48 mag/day)"#report['at_report']['3']['remarks'] = 'Nova candidate in M31 (g-rise > 0.16+-0.01 mag/day, r-rise > 0.24+-0.01 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acxrihd '

In [ ]:
# note that the space at the end is necessary
#report['at_report']['3']['remarks'] = 'Nova candidate in M31 (r-rise > 0.43+-0.01 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF20acgigfo '

## Display report

In [ ]:
print(json.dumps(tns_report, indent=4, sort_keys=True))

## Look for fast risers

In [ ]:
for i in tns_report["at_report"]:
    if tns_report["at_report"][i]["remarks"][:5] == "Early":
        try:
            rise = float(re.findall("rise\s>\s(.*?)\+-.*?\smag", tns_report["at_report"][i]["remarks"])[0])
            if rise >= 0.25:
                print(tns_report["at_report"][i]["remarks"])
        except:
            rise = float(re.findall("rise\s>\s(.*?)\s", tns_report["at_report"][i]["remarks"])[0])
            if rise >= 0.25:
                print(tns_report["at_report"][i]["remarks"])

## Save report

In [ ]:
json_report = '%s/%s.json' % (candidates_dir, refstring)
with open(json_report, 'w') as outfile:
    json.dump(tns_report, outfile)

## Send to TNS

**WARNING: key is used to send candidates!**

In [ ]:
json_report

In [ ]:
r = alerce.send_json_report(api_key, url_tns_api, json_report)

## Show output

In [ ]:
r.content

## Show feedback

In [ ]:
# function for getting reply from report                             
def reply(url, report_id):                                           
    try:                                                               
        # url for getting report reply                                   
        reply_url=url+'/bulk-report-reply'                               
        # construct list of (key,value) pairs                                
        reply_data=[('api_key',(None, api_key)),                         
                    ('report_id',(None,report_id))]                      
        # send report ID using request module                            
        response=requests.post(reply_url, files=reply_data, headers=alerce.tns_headers)              
        # return response                                                
        return response                                                  
    except Exception as e:                                             
        return [None,'Error message : \n'+str(e)]  

In [ ]:
try:
    feedback = reply(url_tns_api, r.json()["data"]["report_id"]).json()["data"]["feedback"]
    print(feedback)
except:
    print("Report not available, probably being processed. Please try again...")

## Check the following link and confirm that the latest SNe are from the reported day

https://www.wis-tns.org/search?&discovered_period_value=1&discovered_period_units=days&reporting_groupid[]=74&discovery_data_source_id=48&num_page=50&format=html&edit[type]=&edit[objname]=&edit[id]=&display[redshift]=1&display[hostname]=1&display[host_redshift]=1&display[source_group_name]=1&display[classifying_source_group_name]=1&display[discovering_instrument_name]=0&display[classifing_instrument_name]=0&display[programs_name]=0&display[internal_name]=1&display[isTNS_AT]=0&display[public]=1&display[end_pop_period]=0&display[spectra_count]=1&display[discoverymag]=1&display[discmagfilter]=1&display[discoverydate]=1&display[discoverer]=1&display[remarks]=0&display[sources]=0&display[bibcode]=0&display[ext_catalogs]=0&sort=desc&order=discoverydate

# Prepare SkyPortal report

In [ ]:
alerce.candidate_hosts = pd.read_csv("hosts/%s_hosts.csv" % refstring)
alerce.candidate_hosts.set_index("oid", inplace=True)
alerce.candidate_hosts.fillna("NULL", inplace=True)
alerce.candidate_hosts

## Load credentials

In [ ]:
skyportal_url = "http://desi2.lbl.gov:5000/api"
credentials_file = "credentials_skyportal.json"
with open(credentials_file) as jsonfile:
    params = json.load(jsonfile)
token = params["token"]

## Build the report

In [ ]:
skyportal_report = []
counter = 0
for oid in candidates[::-1]: # invert to report old candidates first
    if oid in ["ZTF21abkvttz"]: #ZTF21abhpdgd", "ZTF21abgtlir"]:
        continue
    print("\n", counter, oid)
    sn_report = alerce.do_skyportal_report(skyportal_url, token, oid, reporter, verbose=False, test=False)
    if sn_report:# or oid == "ZTF20abyptpc":
        skyportal_report.append(sn_report)
        counter += 1

## Save report

In [ ]:
with open('%s_skyportal.pickle' % refstring, 'wb') as handle:
    pickle.dump(skyportal_report, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('%s_skyportal.pickle' % refstring, 'rb') as handle:
    skyportal_report = pickle.load(handle)
    
display(skyportal_report)

## Submit to SkyPortal

In [ ]:
for report in skyportal_report:
    print(report["candidates"]["id"])
    alerce.send_skyportal_report(skyportal_url, token, report)

## Check SkyPortal

http://desi2.lbl.gov:5000/candidates